# Session 4: Advanced Features of MDAnalysis

<a id='trajanalysis'></a>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

This notebook is adapted from materials developed for the [2018 Workshop/Hackathon](https://github.com/MDAnalysis/WorkshopHackathon2018) and the MDAnalysis [UserGuide](https://userguide.mdanalysis.org/stable/).

## Learning Outcomes


This notebook contains examples of additional features of MDAnalysis that may be useful to you.
You don't have to work through every section in this workshop - pick those that are most of interest to you!

 - [Distance calculations](#distance_calculations)
 - [Creating and modifying Universes](#universes)
 - [Auxiliary](#auxiliary)
 - [Transfromations](#transformations)


#### Additional resources

 - During the workshop, feel free to ask questions at any time
 - For more on how to use MDAnalysis, see the [User Guide](https://userguide.mdanalysis.org/stable/) and [documentation](https://docs.mdanalysis.org/2.0.0-dev0/)
 - Ask questions on the [GitHub Discussions forum](https://github.com/MDAnalysis/mdanalysis/discussions) or on [Discord](https://discord.gg/fXTSfDJyxE)
 - Report bugs on [GitHub](https://github.com/MDAnalysis/mdanalysis/issues?)


# 1. Getting Started

## Google Colab package installs

This installs the necessary packages for Google Colab. *Please only run these if you are using Colab.*

In [ ]:
# NBVAL_SKIP
!pip install condacolab
import condacolab


In [ ]:
# NBVAL_SKIP
import condacolab
condacolab.check()
!mamba install -c conda-forge mdanalysis mdanalysistests mdanalysisdata nglview rdkit

In [ ]:
# NBVAL_SKIP
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

## Initial imports
Let's import MDAnalysis and the other libraries we'll be using in this tutorial.

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

import MDAnalysis as mda
import MDAnalysisData as data
import nglview as nv
import numpy as np
# import matplotlib.pyplot as plt

# 2. Distance calculations with `lib.distance`
<a id='distance_calculations'></a>

This example will show you how to use various functions in `MDAnalysis.lib.distances` to identify hydrogen bonding between certain residues and the water solvent.

A hydrogen bond (in the context of this analysis) will be defined as an interaction between three atoms:
- An acceptor, which is attracting the hydrogen
- A hydrogen, which is being pulled into the acceptor
- A donor, which is bonded to the hydrogen and being dragged along for the ride.

We will use the following geometric criteria:
- a hydrogen-acceptor distance of 3.0A 
- an acceptor-hydrogen-donor angle of greater than 120 degrees.

First, let's load out example dataset from `MDAnalysisData` of a PEG (polyethyleneglycol) chain in water:

In [ ]:
PEG_example = data.datasets.fetch_PEG_1chain()
peg_u = mda.Universe(PEG_example['topology'], PEG_example['trajectory'])

Oxygen atoms in PEG can accept hydrogen bonds from water. Let's make some appropriate selections:

In [ ]:
# select oxygen atoms - types os and oh
acceptors = peg_u.atoms.select_atoms("type os oh")
# select hydrogens from water
hydrogens = peg_u.atoms.select_atoms("type HW")

### Distance criteria

We first want to identify hydrogens and acceptors that are within our distance criteria of 3.0 angstrom.
A naive approach is to use `distance_array`, which will return an array of distances between all acceptors and all hydrogens:

In [ ]:
%%time
distances = mda.lib.distances.distance_array(acceptors.positions, hydrogens.positions, box=peg_u.dimensions)

We can see that the returned distance array has shape (number of acceptors) x (number of water hydrogens):

In [ ]:
print(distances.shape)
print(len(acceptors))
print(len(hydrogens))

The distance in the (i,j) position is the distance between the *i*th acceptor and *j*th hydrogen. We can now find the pairs of indicies with distances in the array that fall below our cutoff. `np.where` is a handy function for returning the indicies where a condition is true:

In [ ]:
acceptor_idx, hydrogen_idx = np.where(da < 3.0)
print(acceptor_idx)
print(hydrogen_idx)

### Using `capped_distance`

This is a great example of where we're not interested in all distances, but instead only those up to a given cutoff - using `capped_distance` is much quicker here!

<div class="alert alert-info"><b>Reminder</b> 

The output of `capped_distance` is no longer a matrix, but arrays of indices and the distance values at those indices.  This can be thought of as a sparse matrix.
 </div>

In [ ]:
%%time 
idx, dists = mda.lib.distances.capped_distance(acceptors.positions, hydrogens.positions, max_cutoff=3.0,
                                               box=peg_u.dimensions)

Notice the time taken compared to `distance_array` above. Try experimenting with the cutoff distance to see how the time required varies. Can you think of other situations where we'd want to use `capped_distance` over `distance_array`? What about the reverse?

**Tip**: `capped_distance` has some other options we aren't using here:
 - You can also define a `min_cutoff`
 - By adding the argument `return_distances=False` to `capped_distance`, we can return only the *indices* without the accompanying distances.

The `idx` array is a `(n, 2)` array of indices. The indicies of our potential hydrogen-bonding acceptors will be in the first column, and the hydrogens in the second column:

In [ ]:
acceptor_idx = idx[:, 0]
hyrogen_idx = idx[:, 1]

print(acceptor_idx)
print(hydrogen_idx)

Compare these to the above output from `distance_array` - did we get the same result?

Remembering that we can slice `AtomGroup`s with numpy arrays, we can use these indices arrays to slice our original `AtomGroup`s to filter them down to only atoms within the distance criterion:

In [ ]:
potential_hbond_acceptors = acceptors[acceptor_idx]
potential_hbond_hydrogens = hydrogens[hydrogen_idx]

To get the **donors** for each hydrogen bond is slightly trickier.
We can use the fact that hydrogens will only have one covalent bond, and simply loop over the hydrogen atoms, grabbing the first (and only) bonded atom of each. 

In [ ]:
potential_hbond_donors = sum(h.bonded_atoms[0] for h in potential_hbond_hydrogens)

<div class="alert alert-info"><b>Reminder</b>  

A `sum()` over `MDAnalysis.Atom` objects will produce an `AtomGroup`!

</div>

### Checking the angle criteria

Now that we've identified hydrogens and acceptors which are close enough for a hydrogen bond, we can now check our angular criteria.
The angle formed by the acceptor-hydrogen-donor must be greater than 120 degrees!

<div class="alert alert-info"><b>Reminder</b>  
    
The input to `calc_angles` must be in the correct order, with the second array of positions being the vertex of the angle.  Results are returned in radians!
 </div>

By first checking the distance criteria and filtering down our input, we greatly reduce the number of angles we must calculate.
This is important as angles calculations are computationally more expensive than distance calculations.

In [ ]:
angles = np.rad2deg(
    mda.lib.distances.calc_angles(potential_hbond_acceptors.positions,
                                  potential_hbond_hydrogens.positions,
                                  potential_hbond_donors.positions, box=peg_u.dimensions)
)

Again we can use `np.where` to get the *indices* of where a condition is True, here if a value is above 120.

In [ ]:
angle_idx = np.where(angles >= 120.0)

Finally, we can slice our list of candidate atoms with `angle_idx` to get three AtomGroups, each representing a different component in a hydrogen bond.

In [ ]:
hbond_acceptors = potential_hbond_acceptors[angle_idx]
hbond_hydrogens = potential_hbond_hydrogens[angle_idx]
hbond_donors = potential_hbond_donors[angle_idx]

In [ ]:
hbond_acceptors

<div class="alert alert-success"> <b>Exercise</b> 

Now let's wrap all the above into a function to analyse hydrogen bonds. 
    
The function will need as input the `hydrogens` and `acceptors` `AtomGroup`s, and return `hbond_acceptors`, `hbond_hydrogens`, `hbond_donors`.

</div>

In [ ]:
def hbonds(hydrogens, acceptors):
    
    """ this function calculates hydrogen bonds """
    
    acc_idx, hyd_idx = idx.T
    
    idx, dists = mda.lib.distances.capped_distance(acceptors.positions, 
                                                   hydrogens.positions, 
                                                   max_cutoff=3.0,
                                                   box=acceptors.dimensions)    

    
    acc_idx, hyd_idx = idx.T

    # select potential hydrogen bonds to check angles
    potential_hbond_acceptors = acceptors[acc_idx]
    potential_hbond_hydrogens = hydrogens[hyd_idx]

    # select hydrogen bond donors by looping over hydrogens and selecting the bonded oxygens
    potential_hbond_donors = sum(h.bonded_atoms[0] for h in potential_hbond_hydrogens)
    
    angles = mda.lib.distances.calc_angles(potential_hbond_acceptors.positions,
                                  potential_hbond_hydrogens.positions,
                                  potential_hbond_donors.positions, 
                                  box=u.dimensions)
    #convert to degrees
    angles = np.rad2deg(angles)
    
    #check angles are larger than 120 degrees
    angle_idx = np.where(angles >= 120.0)
    
    hbond_acceptors = potential_hbond_acceptors[angle_idx]
    hbond_hydrogens = potential_hbond_hydrogens[angle_idx]
    hbond_donors = potential_hbond_donors[angle_idx]
    
    return hbond_acceptors, hbond_hydrogens, hbond_donors

<div class="alert alert-info"> <b>RECAP</b> 
    
How to calculate quickly all possible distances between `AtomGroups` 

 - `self_distance` array only takes one atomgroup
 - `distance_array` takes two atomgroups and they don't have to contain the same number of atoms.
 - `capped_distance` and `self_capped_distance` only considers atoms within a certain distance threshold.
 </div>

# 3. Creating and modifying Universes
<a id='universes'></a>

In this example, we're going create a solvent system and to embed a protein into, in order to demonstrate how to create and modify Universes - but note that MDAnalysis is likely not the best tool for solvating a system!

## Creating a blank Universe

The Universe.empty() method creates a blank Universe. The `natoms` argument must be included. Optional arguments are:

- n_residues (int): number of residues
- n_segments (int): number of segments
- atom_resindex (list): list of resindices for each atom
- residue_segindex (list): list of segindices for each residue
- trajectory (bool): whether to attach a MemoryReader trajectory (default False)
- velocities (bool): whether to include velocities in the trajectory (default False)
- forces (bool): whether to include forces in the trajectory (default False)

Let's create a Universe with 1000 water molecules:

In [ ]:
n_residues = 1000
n_atoms = n_residues * 3

# create resindex list - mapping each atom to a residue. There are three atoms per residue.
resindices = np.repeat(range(n_residues), 3)
assert len(resindices) == n_atoms
print("resindices:", resindices[:10])

# all water molecules belong to 1 segment
segindices = [0] * n_residues
print("segindices:", segindices[:10])

In [ ]:
# create the Universe
sol = mda.Universe.empty(n_atoms,
                         n_residues=n_residues,
                         atom_resindex=resindices,
                         residue_segindex=segindices,
                         trajectory=True) # necessary for adding coordinates
sol

## Adding topology attributes

There isn’t much we can do with our current Universe because MDAnalysis has no information on the particle elements, positions, etc. We can add relevant information manually using TopologyAttrs. For example, to assign `name`s for each atom (deciding on an order of oxygen-hydrogen-hydrogen for each water residue), we can use:

In [ ]:
# names
sol.add_TopologyAttr('name', ['O', 'H1', 'H2']*n_residues)
sol.atoms.names

We can also assign `resname`s to each residue. Let's given them then name 'SOL':


In [ ]:
# residue names
sol.add_TopologyAttr('resname', ['SOL']*n_residues)
sol.atoms.resnames

Notice that when adding `name`s, a property of the atom, we provided a list of 3 * n_residue = n_atoms initial values, while for `resname`, a property of the residue, the initial value list has n_residue values.

Attributes can only be added this way if they've been 'established' - see the list in the [User Guide](https://userguide.mdanalysis.org/stable/topology_system.html). It is possible to add custom attributes, as long as you define them first. See the section on adding custom topology attributes below!

### Exercise

Now see if you can assign:
1. the `type` (ie. element) for each atom (use 'O' for each oxygen and 'H' for each hydrogen)
2. the `resid` of each residue (this should go from 1 to n_residues)
3. the `segid` (ie segment name) that our water atoms belong to (let's call it 'SOL'). **Hint:** `segid` is a segment property. We assigned all waters to the same segment, so there is only one segment in the Universe.


In [ ]:
# elements
sol.add_TopologyAttr('type', ['O', 'H', 'H']*n_residues)
print(sol.atoms.types)

# residue counter
sol.add_TopologyAttr('resid', list(range(1, n_residues+1)))
print(sol.atoms.resids)

# segment/chain names
sol.add_TopologyAttr('segid', ['SOL'])
print(sol.atoms.segids)

## Adding positions
Positions can simply be assigned, without having to add a topology attribute.

The O-H bond length in water is around 0.96 Angstrom, and the bond angle is 104.45°. To give our waters coordinates, we can first obtain a set of coordinates for one molecule, and then translate it for every water molecule. We have 1000 atoms, so let's place them in a 10 x 10 x 10 cube, 8 A apart.

In [ ]:
# coordinates obtained by building a molecule in the program IQMol
h2o = np.array([[ 0,        0,       0      ],  # oxygen
                [ 0.95908, -0.02691, 0.03231],  # hydrogen
                [-0.28004, -0.58767, 0.70556]]) # hydrogen

grid_size = 10 # how many waters per side
spacing = 8    # how far apart each water is

coordinates = []

# translating h2o coordinates around a grid
for i in range(n_residues):
    # place the water center at a new point on the grid
    x = spacing * (i % grid_size)
    y = spacing * ((i // grid_size) % grid_size)
    z = spacing * (i // (grid_size * grid_size))
    xyz = np.array([x, y, z])

    # translate the above wate coordinates to the new center
    coordinates.extend(h2o + xyz.T)


coord_array = np.array(coordinates)
# check, our coord array should have shape (n_atoms, 3) (ie 3000, 3)
print(coord_array.shape)
# check what the first 10 coordinates look like
print(coord_array[:10])

# now we assign the positions
sol.atoms.positions = coord_array


Let's view the system with NGLView:

In [ ]:
sol_view = nv.show_mdanalysis(sol)
sol_view.add_representation('ball+stick', selection='all')
sol_view.center()
sol_view

## Merging Universes

Now we can merge the water with a protein to create a combined system by using MDAnalysis.Merge to combine AtomGroup instances.

We'll use the protein is adenylate kinase (AdK), a phosphotransferase enzyme, from the MDAnalysis datafiles.

In [ ]:
from MDAnalysis.tests.datafiles import PDB_small
protein = mda.Universe(PDB_small)

In [ ]:
protein_view = nv.show_mdanalysis(protein)
protein_view

We will translate the centers of both systems to the origin, so they can overlap in space.

In [ ]:
cog = sol.atoms.center_of_geometry()
print('Original solvent center of geometry: ', cog)
sol.atoms.positions -= cog
cog2 = sol.atoms.center_of_geometry()
print('New solvent center of geometry: ', cog2)

In [ ]:
cog = protein.atoms.center_of_geometry()
print('Original solvent center of geometry: ', cog)
protein.atoms.positions -= cog
cog2 = protein.atoms.center_of_geometry()
print('New solvent center of geometry: ', cog2)

And now, we use `Merge` to combine the two Universes into a new Universe:

In [ ]:
combined = mda.Merge(protein.atoms, sol.atoms)

In [ ]:
combined_view = nv.show_mdanalysis(combined)
combined_view.add_representation("ball+stick", selection="not protein")
combined_view

Unfortunately, some water molecules overlap with the protein. We can create a new AtomGroup containing only the molecules where every atom is further away than 6 angstroms from the protein.

In [ ]:
no_overlap = combined.select_atoms("same resid as (not around 6 protein)")

We can also use `Merge` to construct a new Universe from this AtomGroup:

In [ ]:
merge_u = mda.Merge(no_overlap)

In [ ]:
no_overlap_view = nv.show_mdanalysis(merge_u)
no_overlap_view.add_representation("ball+stick", selection="not protein")
no_overlap_view

Note that a merged Universe will only retain any atom attributes which *both* Universes have. Out protein has masses; but since we didn't add masses when we created the solvent, our new merged universe is also missing masses.

In [ ]:
protein.atoms.masses

In [ ]:
sol.atoms.masses

In [ ]:
merge_u.atoms.masses

## Adding dummy atoms to a Universe

We can also use `mda.Merge` and `mda.Empty` to "extend" a Universe with dummy atoms!

### Exercise

Use what you learnt above to create a Universe containing a single dummy atom (give it the `name` and `resname` 'DUMMY') at the venter of geometry of the protein.

In [ ]:
# Let's create a Universe for our dummy atom
dummy_u = mda.Universe.empty(n_atoms=1, trajectory=True)
dummy_u.add_TopologyAttr('names', ['DUMMY'])
dummy_u.add_TopologyAttr('resname', ['DUMMY'])
dummy_u.positions = merge_u.select_atoms("protein").center_of_geometry()

# Now let's merge it without Universe above! 
new_u = mda.Merge(merge_u.atoms, dummy_u.atoms)

# And finally view our new Universe
dummy_view = nv.show_mdanalysis(new_u)
dummy_view.add_representation("ball+stick", selection="not protein and not water")
dummy_view

## Adding custom attributes

It is possible to add custom topology attributes. First, we need to define a new `AtomAttr`, which has several special fields that must also be defined. Here, we're creating the attribute 'bounciness' (or 'bouncy'), a property of atoms with boolean value. 

In [ ]:
from MDAnalysis.core.topologyattrs import AtomAttr

class Bounciness(AtomAttr):
    dtype=bool
    attrname='bounciness'
    singular='bouncy'
    per_object='atom'

Now we can add this attribute to a universe using `add_TopologyAttr` - this time passing in the class we just defined. We pass the initial values of 'bouncy' (in this case, False (0) for each atom) to the class: 

In [ ]:
from MDAnalysis.tests.datafiles import GRO
u = mda.Universe(GRO)

u.add_TopologyAttr(Bounciness([0]*len(u.atoms)))

We can now access the bounciness property of AtomGroups, and use it as a keyword in atom selections!

In [ ]:
print(u.atoms.bounciness)
u.select_atoms('bouncy')

Currently, all our atoms have bounciness 'False' so an atomselection on 'bouncy' returns no atoms. Let's now set every second atom to be bouncy:

In [ ]:
u.atoms[::2].bounciness = True

Now we can see that the keyword 'bouncy' selects every second atom!

In [ ]:
print(u.select_atoms('bouncy'))

# 4. Auxiliary Readers

# 5. Transformations
<a id='transformations'></a>

## Example: centering a protein in the box

In this example, we're going to go through the process of making a protein whole and centering it in a box (using adenylate kinase (AdK), a phosophotransferase enzyme). 

Calculating bond lengths without considering periodic boundary conditions shows us that there is apparently a bond with a length of 79A! This is obviously caused by the two atoms being in separate periodic images.

In [ ]:
from MDAnalysis.tests.datafiles import TPR, XTC
u = mda.Universe(TPR, XTC, in_memory=True)

u.atoms.bonds.values(pbc=False).max()

We can view the system to confirm this:

In [ ]:
view = nv.show_mdanalysis(u)
view.add_representation('point', 'resname SOL')
view.center()
view

- **Note:** For the step-by-step transformations, we need to load the trajectory into memory so that our changes to the coordinates persist. If your trajectory is too large for that, see the on-the-fly transformation section for how to do this out-of-memory.

### 1. Unwrapping the protein

The first step is to “unwrap” the protein from the border of the box, to make the protein whole. MDAnalysis provides the AtomGroup.unwrap function to do this easily. **Note:** this function requires your universe to have bonds in it.

We loop over the trajectory to unwrap for each frame.

In [ ]:
protein = u.select_atoms('protein')

for ts in u.trajectory:
    protein.unwrap(compound='fragments')

As you can see, our maximum bond length is now much more sensible, and the protein is now whole, but not centered.

In [ ]:
print(u.atoms.bonds.values(pbc=False).max())

unwrapped = nv.show_mdanalysis(u)
unwrapped.add_representation('point', 'resname SOL')
unwrapped.center()
unwrapped

### Aside: `guess_bonds`

As noted above, `wrap` and `unwrap` only work if your Universe has bonds.

In [ ]:
from MDAnalysis.tests.datafiles import GRO
u_nobond = mda.Universe(GRO) 

u_nobond.atoms.unwrap(reference='cog')

So what if this is the case for you? MDAnalysis has a `guess_bonds` feature that will guess the bonds in your system based on their distances and VdW radii!

You can set `guess_bonds`to `True` when loading a Universe, or apply it to an AtomGroup. Note that VdW radii are needed - MDAnalysis will have these values in most cases, but in this example we get an error due to missing radii for a 'DUMMY' atom:

In [ ]:
u_nobond.atoms.guess_bonds()

In this system, DUMMY refers to the 4th dummy atom included in the TIP4P water model. Let's set the VdW radii for these atoms to 0; we can pass this to `guess_bonds` as a dictionary, and now we'll be able to run `unwrap` on our system:

In [ ]:
u_nobond.atoms.guess_bonds({'DUMMY': 0})
u_nobond.atoms.unwrap(reference='cog')

### 2. Centering in the box

The next step is to center the protein in the box. We calculate the center-of-mass of the protein and the center of the box for each timestep. We then use `translate` on an AtomGroup - in this case all atoms - to move all the atoms so that the protein center-of-mass is in the center of the box.

In [ ]:
for ts in u.trajectory:
    protein_center = protein.center_of_mass(wrap=True)
    dim = ts.triclinic_dimensions
    box_center = np.sum(dim, axis=0) / 2
    u.atoms.translate(box_center - protein_center)

The protein is now in the center of the box - but this doesn't seem the case when visualising. This is because we translated the solvent along with the water - some of it is now outside the box.

In [ ]:
centered = nv.show_mdanalysis(u)
centered.add_representation('point', 'resname SOL')
centered.center()
centered

### 3. Wrapping the solvent back into the box
Luckily, MDAnalysis also has AtomGroup.wrap to wrap molecules back into the box. 

In [ ]:
not_protein = u.select_atoms('not protein')

for ts in u.trajectory:
    not_protein.wrap(compound='residues')

Now our protein is properly centered!

In [ ]:
wrapped = nv.show_mdanalysis(u)
wrapped.add_representation('point', 'resname SOL')
wrapped.center()
wrapped

## Doing all this on-the-fly

Running all the transformations above can be difficult if your trajectory is large, or your computational resources are limited. Use on-the-fly transformations to keep your data out-of-memory.

Let's use on-the-fly transformations to repeat the above transformations we performed above. First we re-load our universe:

In [ ]:
import MDAnalysis.transformations as trans

# first, reload the universe
u2 = mda.Universe(TPR, XTC)

protein2 = u2.select_atoms('protein')
not_protein2 = u2.select_atoms('not protein')

Some common transformations are defined in `MDAnalysis.transformations` - see the [documentation](https://docs.mdanalysis.org/stable/documentation_pages/trajectory_transformations.html#currently-implemented-transformations) for what's available.

This includes `wrap`/`unwrap` functions that correspond to those we used above, as well as a `center_in_box` function. We can add these to the trajectory, in the same order as we performed above, using `add_transformations`. **Note:** you can only use `add_transformations` once on a Universe!

In [ ]:
transforms = [trans.unwrap(protein2),
              trans.center_in_box(protein2, wrap=True),
              trans.wrap(not_protein2)]

u2.trajectory.add_transformations(*transforms)

We can visualise to check the transformation is working!

In [ ]:
otf = nv.show_mdanalysis(u2)
otf.add_representation('point', 'resname SOL')
otf.center()
otf

The transformations above require extra information from the Univers in the form of AtomGroups on which the transformations are applied, so they are added after the Unvierse is loaded.
For some transformations - where extra information like this is not required - you can add the transfromation when you load the Universe:

In [ ]:
u3 = mda.Universe(TPR, XTC, transofmrations=[trans.translate([0,0,1])])

## Custom transformations

It's possible to define your own transformation by wriing a function that accepts a Timestep as input, and returns an Timestep. For example, to add a transformation that moves all atoms up along the z axis by 2:

In [ ]:
def up_by_2(ts):
    """Translates atoms up by 2 angstrom"""
    ts.positions += np.array([0.0, 0.0, 0.2])
    return ts

u = mda.Universe(TPR, XTC, transformations=[up_by_2])

If your transformation needs other arguments, you will need to wrap your core transformation with a wrapper function that can accept the other arguments.

In this example, our tranformation will move all atom positions up along the z axis by a specified amount:

In [ ]:
def up_by_x(x):

    """Translates atoms up by x angstrom"""

    def wrapped(ts):

        """Handles the actual Timestep"""

        ts.positions += np.array([0.0, 0.0, float(x)])

        return ts

    return wrapped

u = mda.Universe(TPR, XTC, transformations=[up_by_x(5), up_by_x(2)])

On-the-fly transformation functions can be applied to any property of a Timestep, not just the atom positions! For example, you could use this to define the box dimensions for each frame in a trajectory:

In [ ]:
def set_box(ts):
    ts.dimensions = [10, 20, 30, 90, 90, 90]
    return ts

u = mda.Universe(TPR, XTC, transformations=[set_box])